In [17]:
import os
import re
import glob as gb
import subprocess
import shlex
from packaging import version
import pandas as pd
import numpy as np

In [18]:
THIS_DIR = os.path.abspath('.')
THIS_DIR

'/home/julien/Software/Others/openstudio_gems/OpenStudio-update-gems'

# Find NREL repos that may have a gemspec file

In [91]:
# pip install PyGithub
from github import Github

# First create a Github instance:
g = Github("TOKEN")

## Filter on repos that have 'Ruby' as a language

In [22]:
repo_names = []
ruby_repo_names = []
repo = None
for repo in g.get_organization('NREL').get_repos():
    repo_name = repo.name
    # print(repo_name)
    repo_names.append(repo_name)
    if 'Ruby' in repo.get_languages():
        ruby_repo_names.append(repo_name)

# Loop on all repos that have ruby, parse *.gemspec file

In [23]:
RE_VERSION = re.compile(r'^\s+\w+\.add_(?P<type>(?:.*?_)?dependency)\s+[\'\"](?P<name>.*?)[\'\"]\s*,\s*[\'\"](?P<version>.*?)[\'\"]')

In [24]:
# List of branches that are open and aim to bump dependencies
# If not specified here, will use default branch (eg: master or develop typically)

bump_branches = {
 # 'NREL/openstudio-extension-gem': 'Bump_deps',
 #'NREL/OpenStudio-measure-tester-gem': 'upgrade-dependencies',
 #'NREL/openstudio-common-measures-gem': 'feature/13-openstudio-3.x',
}


gemspecs = {}
for ruby_repo_name in ruby_repo_names:
    repo = g.get_user('NREL').get_repo(ruby_repo_name)
    repo_full_name = repo.full_name
    
    if (repo_full_name in bump_branches):
        b = repo.get_branch(bump_branches[repo_full_name])
        print("For {r}, getting branch {b} at {s}".format(
          r=repo_full_name, b=bump_branches[repo_full_name], s=b.commit.sha))
        content_files = repo.get_contents('.', ref=b.commit.sha)
    else:
        # **Try** to get Bump_deps
        # if 'Bump_deps' in [x.name for x in repo.get_branches()]:
        #    b = repo.get_branch('Bump_deps')
        #    print(f"Bump_deps exists already on {ruby_repo_name}")
        content_files = repo.get_contents('.')
        
    for content_file in content_files:
    
        if os.path.splitext(content_file.name)[1] == '.gemspec':
            gemspecs[repo_full_name] = []
            content = content_file.decoded_content.decode()

            for line in content.splitlines():
                m = RE_VERSION.search(line)
                if m:
                    d = m.groupdict()
                    gemspecs[repo_full_name].append(d)

# Analyze versions

In [25]:
empty_gemspecs = [k for k, v in gemspecs.items() if not v]

In [26]:
for k, v in gemspecs.items():
    for x in v:
        x.update({'gem': k})
        #x.pop('gem')
        
gemspecs_list = []

for k, v in gemspecs.items():
    for x in v:
        gemspecs_list.append(x)

In [27]:
df = pd.DataFrame(gemspecs_list)[['gem', 'name', 'version', 'type']]

## Find Max version for each

### In our gemspecs

In [28]:
df['parse_version'] = df['version'].str.replace('~> ', '').str.replace('<= ', '').str.replace('= ', '').apply(version.parse)
s_max_version = df.groupby('name')['parse_version'].max()
s_max_version

name
activesupport                      4.2.5
aes                                0.5.0
aws-sdk-core                      2.2.37
bcl                                0.6.1
builder                            3.2.4
bundler                              2.1
ci_reporter                        2.0.0
ci_reporter_rspec                  1.0.0
coveralls                         0.8.21
dencity                            0.1.0
docile                               1.1
faraday                            1.0.1
git                                1.6.0
google-api-client                  0.8.6
json-schema                        2.8.0
launchy                          < 2.5.0
minitar                              0.9
minitest                          5.14.0
minitest-ci                       5.10.3
minitest-reporters                 1.4.2
net-scp                            2.0.0
net-ssh                            4.2.0
nokogiri                           1.8.2
octokit                           4.18.0
openstudio-

### On rubygems for Ruby 2.5.5

In [31]:
def find_max_avail_gem_version(gem_name):

    # RE_GEM_AVAIL = re.compile(r'(?P<name>.*?) \((?P<version>.*)\)')
    RE_GEM_AVAIL = re.compile(r'(?P<name>.*?) \((?P<version>\d+\.\d+.\d*)(?P<rest>.*)\)')

    process = subprocess.Popen(shlex.split(f"gem search --remote --no-details --no-prerelease -e {gem_name}"),
                               shell=False,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    out, err = process.communicate()
    errcode = process.returncode
    if errcode == 0:
        max_avail = out.rstrip().decode()
        print(f"found {max_avail}")
        if (m := RE_GEM_AVAIL.match(max_avail)):
            d = m.groupdict()
            if d['rest']:
                print(f"Found extra stuff, not parsed after '{d['version']}': {max_avail}")
            return version.parse(d['version'])
        else:
            #raise ValueError
            print(f"Couldn't match regex for {gem_name}: {max_avail}")
            return None
    else:
        print(out)
        print(err)
        raise ValueError(f"Error for {gem_name}")

In [32]:
s_max_known = pd.Series(s_max_version.index,
                        index=s_max_version.index,
                        name="max_known").apply(find_max_avail_gem_version)

df_max_versions = pd.concat([s_max_version, s_max_known], axis=1)

found activesupport (6.1.1)
found aes (0.5.1)
found aws-sdk-core (3.112.0)
found bcl (0.6.1)
found builder (3.2.4)
found bundler (2.2.8)
found ci_reporter (2.0.0)
found ci_reporter_rspec (1.0.0)
found coveralls (0.8.23)
found dencity (0.1.0)
found docile (1.3.5)
found faraday (1.3.0)
found git (1.8.1)
found google-api-client (0.53.0)
found json-schema (2.8.1)
found launchy (2.5.0, 2.4.3 java)
Found extra stuff, not parsed after '2.5.0': launchy (2.5.0, 2.4.3 java)
found minitar (0.9)
Couldn't match regex for minitar: minitar (0.9)
found minitest (5.14.3)
found minitest-ci (3.4.0)
found minitest-reporters (1.4.3)
found net-scp (3.0.0)
found net-ssh (6.1.0)
found nokogiri (1.11.1 ruby arm64-darwin java x64-mingw32 x86-linux x86-mingw32 x86_64-darwin x86_64-linux, 1.6.1 x86-mswin32-60, 1.4.4.1 x86-mswin32)
Found extra stuff, not parsed after '1.11.1': nokogiri (1.11.1 ruby arm64-darwin java x64-mingw32 x86-linux x86-mingw32 x86_64-darwin x86_64-linux, 1.6.1 x86-mswin32-60, 1.4.4.1 x86-msw

### Difference between the two

In [33]:
df_max_versions[df_max_versions["parse_version"] != df_max_versions["max_known"]]

,parse_version,max_known
name,,
activesupport,4.2.5,6.1.1
aes,0.5.0,0.5.1
aws-sdk-core,2.2.37,3.112.0
bundler,2.1,2.2.8
coveralls,0.8.21,0.8.23
docile,1.1,1.3.5
faraday,1.0.1,1.3.0
git,1.6.0,1.8.1
google-api-client,0.8.6,0.53.0


## Visualize

In [34]:
def check_versions(col):
    """
    col is a pd.Series for a single gem used in our projects
    
    Return:
    * True if no problems
    * False if problems: found more than one required version
    """
    return (col[col.notnull()].nunique() <= 1)

In [35]:
def color_by_type(val):
    if val == 'dependency':
        return 'color: red'
    elif val == 'development_dependency':
        return 'color: orange'
    else:
        return ''

def color_by_dep_type(data):
    return df_piv['type'].loc[data.index, data.columns].applymap(color_by_type)

In [53]:
df.set_index(['name', 'gem'])[df.set_index(['name', 'gem']).index.duplicated(keep=False)]

,,version,type,parse_version
name,gem,,,
parallel_tests,NREL/openstudio-standards,<= 2.32.0,development_dependency,2.32.0
nokogiri,NREL/openstudio-standards,<= 1.6.8.1,development_dependency,1.6.8.1
bundler,NREL/openstudio-standards,~> 1.9,development_dependency,1.9
parallel_tests,NREL/openstudio-standards,~> 3.0.0,development_dependency,3.0.0
nokogiri,NREL/openstudio-standards,<= 1.8.2,development_dependency,1.8.2
bundler,NREL/openstudio-standards,~> 2.1,development_dependency,2.1


In [60]:
df.drop_duplicates?

In [66]:
df.set_index(['gem', 'name']).drop_duplicates?

Object `drop_duplicates` not found.


In [73]:
df.set_index(['gem', 'name']).loc[df.set_index(['gem', 'name']).index.drop_duplicates(keep='last')]

version  \
gem                                           name                                   
NREL/bcl-gem                                  builder                        3.2.4   
                                              faraday                     ~> 1.0.1   
                                              minitar                       ~> 0.9   
                                              openstudio_measure_tester   ~> 0.2.2   
                                              rexml                          3.2.4   
...                                                                            ...   
NREL/openstudio-load-flexibility-measures-gem rake                         ~> 13.0   
                                              rspec                         ~> 3.9   
                                              rubocop                    ~> 0.54.0   
                                              openstudio-extension        ~> 0.3.2   
                                              openstudio-standards       ~> 0.2.12   

                                                                                           type  \
gem                                           name                                                
NREL/bcl-gem                                  builder                                dependency   
                                              faraday                                dependency   
                                              minitar                                dependency   
                                              openstudio_measure_tester              dependency   
                                              rexml                                  dependency   
...                                                                                         ...   
NREL/openstudio-load-flexibility-measures-gem rake                       development_dependency   
                                              rspec                      development_dependency   
                                              rubocop                    development_dependency   
                                              openstudio-extension                   dependency   
                                              openstudio-standards                   dependency   

                                                                        parse_version  
gem                                           name                                     
NREL/bcl-gem                                  builder                           3.2.4  
                                              faraday                           1.0.1  
                                              minitar                             0.9  
                                              openstudio_measure_tester         0.2.2  
                                              rexml                             3.2.4  
...                                                                               ...  
NREL/openstudio-load-flexibility-measures-gem rake                               13.0  
                                              rspec                               3.9  
                                              rubocop                          0.54.0  
                                              openstudio-extension              0.3.2  
                                              openstudio-standards             0.2.12  

[138 rows x 3 columns]

In [76]:
df[['gem', 'name']].drop_duplicates().index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            128, 129, 130, 131, 132, 133, 134, 135, 136, 137],
           dtype='int64', length=135)

In [52]:
df.index.duplicated?

In [47]:
df[df['type'] == 'dependency'].pivot(index='gem', columns='name', values='version')

name,activesupport,aws-sdk-core,bcl,builder,bundler,dencity,docile,faraday,git,minitar,...,rubocop-checkstyle_formatter,rubyzip,semantic,simplecov,simplecov-html,spreadsheet,sshkey,uuid,yamler,zliby
gem,,,,,,,,,,,,,,,,,,,,,
NREL/OpenStudio-analysis-gem,NaN,NaN,~> 0.6.1,NaN,NaN,~> 0.1.0,NaN,~> 1.0.1,NaN,NaN,...,NaN,~> 2.3.0,~> 1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NREL/OpenStudio-aws-gem,NaN,= 2.2.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,~> 1.4,NaN,NaN,NaN,~> 2.0,NaN,NaN,NaN
NREL/OpenStudio-fault-measure-gem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NREL/OpenStudio-measure-tester-gem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,~> 1.6.0,NaN,...,~> 0.4.0,NaN,NaN,~> 0.18.2,NaN,NaN,NaN,NaN,NaN,NaN
NREL/bcl-gem,NaN,NaN,NaN,3.2.4,NaN,NaN,NaN,~> 1.0.1,NaN,~> 0.9,...,NaN,~> 2.3.0,NaN,NaN,NaN,1.2.6,NaN,~> 2.3.9,0.1.0,0.0.5
NREL/haystack_ruby,>= 5.2.4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NREL/openstudio-aedg-gem,NaN,NaN,NaN,NaN,~> 2.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NREL/openstudio-calibration-gem,NaN,NaN,NaN,NaN,~> 2.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NREL/openstudio-common-measures-gem,NaN,NaN,NaN,NaN,~> 2.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# df.pivot(index='name', columns='gem', values='version')

df_piv = df.loc[df[['gem', 'name']].drop_duplicates().index].pivot(index='gem', columns='name', values=['version', 'type'])

In [78]:
df_max_versions

,parse_version,max_known
name,,
activesupport,4.2.5,6.1.1
aes,0.5.0,0.5.1
aws-sdk-core,2.2.37,3.112.0
bcl,0.6.1,0.6.1
builder,3.2.4,3.2.4
bundler,2.1,2.2.8
ci_reporter,2.0.0,2.0.0
ci_reporter_rspec,1.0.0,1.0.0
coveralls,0.8.21,0.8.23


In [79]:
(df_piv['version'].loc[:, ~df_piv['version'].apply(check_versions)].fillna('').style
   .apply(color_by_dep_type, axis=None)
   .set_caption("Gem versions. Red = add_dependency, orange = add_development_dependency (add_runtime_dependency not handled).\nList of branches used (develop otherwise):\n{d}".format(d=bump_branches))
)

name,activesupport,builder,bundler,faraday,openstudio-extension,openstudio-standards,openstudio-workflow,openstudio_measure_tester,parallel,public_suffix,rake,roo,rspec,rubocop,rubocop-checkstyle_formatter,simplecov-html
gem,,,,,,,,,,,,,,,,
NREL/OpenStudio-analysis-gem,,,,~> 1.0.1,,,,,,,~> 13.0,~> 2.8.3,~> 3.9,~> 0.54.0,~> 0.4.0,
NREL/OpenStudio-aws-gem,,,,,,,,,,,~> 12.3,,,,,
NREL/OpenStudio-fault-measure-gem,,,~> 2.1,,~> 0.3.1,~> 0.2.12,,,,,~> 13.0,,~> 3.9,~> 0.54.0,,
NREL/OpenStudio-measure-tester-gem,,,~> 2.1,,,,,,,,~> 13.0,,~> 3.9,~> 0.54.0,~> 0.4.0,
NREL/OpenStudio-workflow-gem,,~> 3.2.4,~> 2.1,,,~> 0.2.12,,,~> 1.19.1,~> 4.0.3,~> 12.3,,~> 3.9,~> 0.80.1,~> 0.4.0,
NREL/bcl-gem,,3.2.4,,~> 1.0.1,,,,~> 0.2.2,,,,,,,,
NREL/bricr,,,~> 1.17.1,,,,,,~> 1.12,,,,,,,
NREL/dencity-gem,,,~> 1.10,,,,,,,,~> 10.0,,,,,
NREL/dencity-scripts,,,~> 1.7,,,,,,,,~> 10.0,,,,,


## Save to HTML for sharing

In [80]:
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])


def getStyles():
    styles = [
        hover(),
        dict(selector="tr:nth-child(2n+1)", props=[('background', '#f5f5f5')]),
        dict(selector="td", props=[("text-align", "center")]),
        dict(selector="caption", props=[("caption-side", "bottom"),
                                        ("color", "grey")])
    ]
    return styles



html = (df_piv['version'].loc[:, ~df_piv['version'].apply(check_versions)].fillna('').style
   .set_table_attributes('style="border:1px solid black;'
                                     'border-collapse:collapse;"')
   .set_properties(**{'border': '1px solid black',
                                  'border-collapse': 'collapse',
                                  'border-spacing': '0px'})
   .apply(color_by_dep_type, axis=None)
   .set_table_styles(getStyles())
   .set_caption("Gem versions. Red = add_dependency, orange = add_development_dependency (add_runtime_dependency not handled).\nList of branches used (develop otherwise):\n{d}".format(d=bump_branches))
).render()

with open('result.html', 'w') as f:
    f.write(html)

# Make changes

## Pickle / Reload (to avoid rerunning everything)

In [81]:
df.to_pickle('df.pickle')
df_max_versions[['parse_version', 'max_known']].to_pickle('df_max_versions.pickle')

### Reload

In [9]:
bump_branches = {
 'NREL/openstudio-extension-gem': 'Bump_deps',
 'NREL/OpenStudio-measure-tester-gem': 'upgrade-dependencies',
 'NREL/openstudio-common-measures-gem': 'feature/13-openstudio-3.x',
}


In [82]:
os.chdir(THIS_DIR)
df = pd.read_pickle('df.pickle')
df_max_versions = pd.read_pickle('df_max_versions.pickle')
df_piv = df.loc[df[['gem', 'name']].drop_duplicates().index].pivot(index='gem', columns='name', values=['version', 'type'])

In [89]:
df[df['type'] == 'dependency']['name'].unique()

array(['builder', 'faraday', 'minitar', 'openstudio_measure_tester',
       'rexml', 'rubyzip', 'spreadsheet', 'uuid', 'yamler', 'zliby',
       'bcl', 'dencity', 'roo', 'semantic', 'aws-sdk-core', 'net-scp',
       'net-ssh', 'sshkey', 'openstudio-extension',
       'openstudio-standards', 'activesupport', 'git', 'minitest',
       'minitest-reporters', 'rake', 'rubocop',
       'rubocop-checkstyle_formatter', 'simplecov', 'simplecov-html',
       'docile', 'bundler', 'octokit', 'openstudio-workflow', 'parallel'],
      dtype=object)

In [90]:
df_max_versions.loc[df[df['type'] == 'dependency']['name'].unique()]

,parse_version,max_known
name,,
builder,3.2.4,3.2.4
faraday,1.0.1,1.3.0
minitar,0.9,None
openstudio_measure_tester,0.2.3,0.2.3
rexml,3.2.4,3.2.4
rubyzip,2.3.0,2.3.0
spreadsheet,1.2.6,1.2.7
uuid,2.3.9,2.3.9
yamler,0.1.0,0.1.0


## HARCODE WANTED VERSIONS

<p style='font-size:20px; color:red;'> README HERE </p>

**Modify the versions you want, ideally you'd know beforehand what the openstudio-xxx gems will be tagged as**

In [12]:
# pd.read_clipboard().set_index('name')['Harcoded'].to_dict()

harcoded_versions = {'activesupport': '~> 6.0',
 'aes': '~> 0.5.0',
 'aws-sdk-core': '~> 3.90.1',
 'bcl': '~> 0.5.8',
 'builder': '~> 3.2.4',
 'bundler': '~> 2.1',
 'ci_reporter': '~> 2.0.0',
 'ci_reporter_rspec': '~> 1.0.0',
 'coveralls': '~> 0.8.23',
 'dencity': '~> 0.1.0',
 'docile': '~> 1.3.2',
 'faraday': '~> 1.0.0',
 'git': '~> 1.6.0',
 'github_api': '~> 0.18.2',
 'google-api-client': '~> 0.37.2',
 'json-schema': '~> 2.8.1',
 'json_pure': '~> 2.2',
 'minitest': '~> 5.14.0',
 'minitest-ci': '~> 3.4.0',
 'minitest-reporters': '~> 1.4.2',
 'net-scp': '~> 2.0.0',
 'net-ssh': '~> 5.2.0',
 'nokogiri': '~> 1.10.9',
 'openstudio-common-measures': '~> 0.2.0',    # TODO: Release Needed
 'openstudio-extension': '~> 0.2.0',          # TODO: Release Needed
 'openstudio-standards': '~> 0.2.11-rc1',         # TODO: not sure what their new version number will be
 'openstudio-workflow': '~> 2.0.0',           # TODO: Official Release Needed
 'openstudio_measure_tester': '~> 0.2.0',    # TODO: needs re-release, perhaps with a different version number
 'parallel': '~> 1.19.1',
 'public_suffix': '~> 4.0.3',
 'rainbow': '~> 3.0.0',
 'rake': '~> 13.0',
 'rest-client': '~> 2.1.0',
 'roo': '~> 2.8.3',
 'rspec': '~> 3.9.0',
 'rubocop': '~> 0.80.1',
 'rubocop-checkstyle_formatter': '~> 0.4.0',
 'rubyXL': '~> 3.4.12',
 'rubyzip': '~> 2.2.0',
 'semantic': '~> 1.6.1',
 'simplecov': '~> 0.18.5',
 'simplecov-html': '~> 0.12.2',
 'sshkey': '~> 2.0',
 'yard': '~> 0.9.24'}

In [19]:
df_max_versions = df_max_versions.join(pd.Series(harcoded_versions, name='Harcoded'))
df_max_versions

,parse_version,max_known,Harcoded
name,,,
activesupport,4.2.5,6.0.2,~> 6.0
aes,0.5.0,0.5.0,~> 0.5.0
aws-sdk-core,2.2.37,3.92.0,~> 3.90.1
bcl,0.5.8,0.5.8,~> 0.5.8
builder,3.2.4,3.2.4,~> 3.2.4
bundler,2.1,2.1.4,~> 2.1
ci_reporter,2.0.0,2.0.0,~> 2.0.0
ci_reporter_rspec,1.0.0,1.0.0,~> 1.0.0
coveralls,0.8.21,0.8.23,~> 0.8.23


## Clone repos, branch, and make changes

In [13]:
# This is the folder in which the repos will be cloned
ROOT_GEMS_DIR = "/home/julien/Software/Others/openstudio_gems/"

<p style='font-size:20px; color:red;'> README HERE </p>

**These are the repos where you would potentially push a new "Bump_deps" branch**

In [85]:
adjust_repos = list(df.loc[:, 'gem'].unique())
adjust_repos

['NREL/bcl-gem',
 'NREL/OpenStudio-analysis-gem',
 'NREL/OpenStudio-aws-gem',
 'NREL/openstudio-standards',
 'NREL/OpenStudio-workflow-gem',
 'NREL/dencity-scripts',
 'NREL/dencity-gem',
 'NREL/OpenStudio-fault-measure-gem',
 'NREL/bricr',
 'NREL/haystack_ruby',
 'NREL/OpenStudio-measure-tester-gem',
 'NREL/simplecov',
 'NREL/openstudio-extension-gem',
 'NREL/openstudio-model-articulation-gem',
 'NREL/openstudio-gems',
 'NREL/openstudio-common-measures-gem',
 'NREL/openstudio-aedg-gem',
 'NREL/openstudio-ee-gem',
 'NREL/openstudio-calibration-gem',
 'NREL/openstudio-metadata-gem',
 'NREL/openstudio-load-flexibility-measures-gem']

In [14]:
adjust_repos = list(df.loc[:, 'gem'].unique())
adjust_repos.remove('NREL/simplecov')
adjust_repos = ['NREL/OpenStudio-analysis-gem',
 'NREL/OpenStudio-aws-gem',
 'NREL/openstudio-standards',
 'NREL/OpenStudio-workflow-gem',
 'NREL/dencity-scripts',
 'NREL/dencity-gem',
 'NREL/bricr',
 'NREL/OpenStudio-measure-tester-gem',
 'NREL/openstudio-extension-gem',
 'NREL/openstudio-model-articulation-gem',
 'NREL/openstudio-gems',
 'NREL/openstudio-common-measures-gem']

In [15]:
adjust_repos = [
 'NREL/openstudio-standards',
 'NREL/openstudio-gems']

In [16]:
bump_branches

{'NREL/openstudio-extension-gem': 'Bump_deps',
 'NREL/OpenStudio-measure-tester-gem': 'upgrade-dependencies',
 'NREL/openstudio-common-measures-gem': 'feature/13-openstudio-3.x'}

In [22]:
for repo_full_name in adjust_repos:
    
    s_gems = df_piv.loc[repo_full_name, 'version']
    #s_gems = df_piv['version'].loc[:, ~df_piv['version'].apply(check_versions)].loc[repo_full_name]
    s_gems = s_gems[s_gems.notnull()]
    this_replace_v = df_max_versions.loc[s_gems.index, 'Harcoded']

    
    os.chdir(ROOT_GEMS_DIR)
    
    repo_name = os.path.basename(repo_full_name)
    repo_path = os.path.join(ROOT_GEMS_DIR, repo_name)
    
    # Clone
    if not os.path.exists(repo_path):
        repo = g.get_repo(repo_full_name)
        git_url = repo.clone_url.replace('.com/', '.com:').replace('https://', 'git@')
        print(f"{repo_name} doesn't exist yet, cloning")

        if not subprocess.call(shlex.split(f"git clone {git_url}")):
            print(f"Failed to clone for {repo_name}: {git_url}")
    
    else:
        os.chdir(repo_path)
        print(f" * {repo_name} exists already")
        process = subprocess.Popen(shlex.split("git fetch --all"),
                               shell=False,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
        out, err = process.communicate()
        print(err)
    
    # Checkout branch
    os.chdir(repo_path)
    
    if repo_full_name in bump_branches:
        branch_name = bump_branches[repo_full_name]
    else:
        branch_name = 'Bump_deps'
    
    process = subprocess.Popen(shlex.split(f"git checkout -b {branch_name}"),
                               shell=False,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    out, err = process.communicate()
    print(err)
    
    process = subprocess.Popen(shlex.split(f"git pull origin {branch_name}"),
                               shell=False,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    out, err = process.communicate()
    print(err)
    
    # Find gemspec file
    gemspec_files = gb.glob("*.gemspec")
    if len(gemspec_files) != 1:
        print(f"Found more than one gemspec file for {repo_name}: {gemspec_files}")
        continue
    gemspec_file = gemspec_files[0]
    
    
    with open(gemspec_file, 'r') as f:
        content = f.read()

    lines = content.splitlines()

    RE_VERSION_REPLACE = re.compile(r'(?P<add>^\s+\w+\.add_.*?dependency)\s+[\'\"](?P<name>.*?)[\'\"]\s*,\s*[\'\"](?P<version>.*?)[\'\"]')
    new_lines = []

    for line in lines:
        m = RE_VERSION.search(line)
        if m:
            gem_name = m.groupdict()['name']
            if gem_name in df_max_versions.index:
                #new_version = df_max_versions.loc[gem_name, 'max_version']
                #repl_version = f"\g<add> '\g<name>', '~> {new_version}'"
                # USE HARDCODED ONE
                new_version = df_max_versions.loc[gem_name, 'Harcoded']
                repl_version = f"\g<add> '\g<name>', '{new_version}'"
                new_lines.append(RE_VERSION_REPLACE.sub(repl_version, line))
            else:
                print(f"{gem_name} not found in df_max_versions, ignoring")
                new_lines.append(line)
        else:
            new_lines.append(line)
    
    with open(gemspec_file, 'w') as f:
        f.write("{}\n".format("\n".join(new_lines)))
    
    subprocess.call(shlex.split(f"git add {gemspec_file}"))
    subprocess.call(shlex.split(f"git commit -m 'Bump dependencies (OpenStudio-update-gems)'"))
    process = subprocess.Popen(shlex.split(f"git push --set-upstream origin {branch_name}"),
                               shell=False,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    out, err = process.communicate()
    if not process.returncode == 0:
        print(out, err)
        
os.chdir(THIS_DIR)

 * openstudio-standards exists already
b''
b"fatal: A branch named 'Bump_deps' already exists.\n"
b"fatal: couldn't find remote ref Bump_deps\n"
b'' b'ERROR: Permission to NREL/openstudio-standards.git denied to jmarrec.\nfatal: Could not read from remote repository.\n\nPlease make sure you have the correct access rights\nand the repository exists.\n'
 * openstudio-gems exists already
b'From github.com:NREL/openstudio-gems\n   bc2837f..366bffd  develop          -> origin/develop\n * [new branch]      nokogiri         -> origin/nokogiri\n * [new branch]      standards_update -> origin/standards_update\n * [new branch]      workflow_update  -> origin/workflow_update\n'
b"Switched to a new branch 'Bump_deps'\n"
b"fatal: couldn't find remote ref Bump_deps\n"
b'' b'ERROR: Permission to NREL/openstudio-gems.git denied to jmarrec.\nfatal: Could not read from remote repository.\n\nPlease make sure you have the correct access rights\nand the repository exists.\n'


### Cleanup current status